# Numeric Computation Agent Demo

This notebook demonstrates how the Numeric Computation Agent works using OpenAI's Agents SDK. The agent can perform mathematical computations including factorial, exponential, and Fibonacci calculations using tool calling.

## 1. Install Required Dependencies

First, we need to install the necessary packages.

In [ ]:
!pip install openai-agents==0.5.1
!pip install openai==2.8.0
!pip install python-dotenv==1.2.1

## 2. Import Libraries and Set Up Environment

Import the required libraries from the OpenAI Agents SDK.

In [7]:
import os
from agents import Agent, Runner, function_tool
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 3. Configure API Key

Set up the OpenAI API key. Make sure you have your API key in a `.env` file or set it directly.

In [8]:
# Set the OpenAI API key from environment variable
# Make sure you have OPENAI_API_KEY in your .env file or uncomment the line below
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ Warning: OPENAI_API_KEY not found in environment variables")
else:
    print("✓ OpenAI API key loaded successfully")

✓ OpenAI API key loaded successfully


## 4. Define Mathematical Tools with Print Statements

Create tool functions for mathematical operations. Each tool will print its input parameters and output results when called.

In [9]:
@function_tool
def compute_factorial(n: int) -> dict:
    """Calculates the factorial of a non-negative integer.
    
    Args:
        n (int): The non-negative integer to compute factorial for
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_factorial")
    print(f"   INPUT: n = {n}")
    
    if n < 0:
        result = {
            "status": "error",
            "error_message": "Factorial is not defined for negative numbers."
        }
        print(f"   OUTPUT: {result}")
        return result
    
    if n > 170:
        result = {
            "status": "error",
            "error_message": "Number too large. Factorial computation limited to n <= 170."
        }
        print(f"   OUTPUT: {result}")
        return result
    
    try:
        factorial_result = 1
        for i in range(2, n + 1):
            factorial_result *= i
        
        result = {
            "status": "success",
            "result": f"The factorial of {n} is {factorial_result}"
        }
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {
            "status": "error",
            "error_message": f"Error computing factorial: {str(e)}"
        }
        print(f"   OUTPUT: {result}")
        return result


@function_tool
def compute_exponential(base: float, exponent: float) -> dict:
    """Calculates base raised to the power of exponent.
    
    Args:
        base (float): The base number
        exponent (float): The exponent
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_exponential")
    print(f"   INPUT: base = {base}, exponent = {exponent}")
    
    try:
        result_value = base ** exponent
        
        if abs(result_value) > 1e308:
            result = {
                "status": "error",
                "error_message": "Result too large to compute."
            }
            print(f"   OUTPUT: {result}")
            return result
        
        result = {
            "status": "success",
            "result": f"{base} raised to the power of {exponent} is {result_value}"
        }
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {
            "status": "error",
            "error_message": f"Error computing exponential: {str(e)}"
        }
        print(f"   OUTPUT: {result}")
        return result


@function_tool
def compute_fibonacci(n: int) -> dict:
    """Calculates the nth Fibonacci number (0-indexed).
    
    Args:
        n (int): The position in the Fibonacci sequence (0-indexed)
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_fibonacci")
    print(f"   INPUT: n = {n}")
    
    if n < 0:
        result = {
            "status": "error",
            "error_message": "Fibonacci is not defined for negative positions."
        }
        print(f"   OUTPUT: {result}")
        return result
    
    if n > 1000:
        result = {
            "status": "error",
            "error_message": "Position too large. Fibonacci computation limited to n <= 1000."
        }
        print(f"   OUTPUT: {result}")
        return result
    
    try:
        if n == 0:
            fib_result = 0
        elif n == 1:
            fib_result = 1
        else:
            a, b = 0, 1
            for _ in range(2, n + 1):
                a, b = b, a + b
            fib_result = b
        
        result = {
            "status": "success",
            "result": f"The {n}th Fibonacci number is {fib_result}"
        }
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {
            "status": "error",
            "error_message": f"Error computing Fibonacci: {str(e)}"
        }
        print(f"   OUTPUT: {result}")
        return result

print("✓ Mathematical tools defined successfully")

✓ Mathematical tools defined successfully


## 5. Initialize the Agent

Create the Numeric Computation Agent with the defined tools and instructions.

In [10]:
# Create the agent with the mathematical tools
agent = Agent(
    name="Numeric Computation Agent",
    instructions=(
        "You are a numeric computation specialist. You can calculate factorial, "
        "exponential (power), and Fibonacci numbers. "
        "When the user asks for any of these computations, use the appropriate tool. "
        "For factorial, use `compute_factorial`. "
        "For exponential (power), use `compute_exponential`. "
        "For Fibonacci, use `compute_fibonacci`. "
        "If the user asks about anything else, politely inform them that your expertise "
        "is limited to these numeric computations."
    ),
    model="gpt-5-mini",  # You can also use other models
    tools=[compute_factorial, compute_exponential, compute_fibonacci],
)

print("✓ Agent initialized successfully")

✓ Agent initialized successfully


## 6. Execute Agent with Tool Usage

Let's test the agent by asking it to compute the factorial of 50. You'll see the tool being called with its input and output.

In [11]:
# Run the agent with a query
query = "Compute the factorial of 50"
print(f"📝 Query: {query}")
print("=" * 60)

result = await Runner.run(agent, input=query)

print("=" * 60)
print(f"🤖 Agent Response:")
print(result.final_output)

📝 Query: Compute the factorial of 50
🔧 TOOL CALLED: compute_factorial
   INPUT: n = 50
   OUTPUT: {'status': 'success', 'result': 'The factorial of 50 is 30414093201713378043612608166064768844377641568960512000000000000'}
🤖 Agent Response:
50! = 30414093201713378043612608166064768844377641568960512000000000000


## 7. Try More Examples

Let's test other computations to see all tools in action.

In [12]:
# Test exponential computation
query2 = "What is 2 raised to the power of 10?"
print(f"📝 Query: {query2}")
print("=" * 60)

result2 = await Runner.run(agent, input=query2)

print("=" * 60)
print(f"🤖 Agent Response:")
print(result2.final_output)

📝 Query: What is 2 raised to the power of 10?
🔧 TOOL CALLED: compute_exponential
   INPUT: base = 2.0, exponent = 10.0
   OUTPUT: {'status': 'success', 'result': '2.0 raised to the power of 10.0 is 1024.0'}
🤖 Agent Response:
2^10 = 1024.


In [13]:
# Test Fibonacci computation
query3 = "What is the 15th Fibonacci number?"
print(f"📝 Query: {query3}")
print("=" * 60)

result3 = await Runner.run(agent, input=query3)

print("=" * 60)
print(f"🤖 Agent Response:")
print(result3.final_output)

📝 Query: What is the 15th Fibonacci number?
🔧 TOOL CALLED: compute_fibonacci
   INPUT: n = 15
   OUTPUT: {'status': 'success', 'result': 'The 15th Fibonacci number is 610'}
🤖 Agent Response:
The 15th Fibonacci number is 610.


In [14]:
# Test multiple computations in one query
query4 = "Calculate the factorial of 7 and also tell me what is 3 to the power of 4"
print(f"📝 Query: {query4}")
print("=" * 60)

result4 = await Runner.run(agent, input=query4)

print("=" * 60)
print(f"🤖 Agent Response:")
print(result4.final_output)

📝 Query: Calculate the factorial of 7 and also tell me what is 3 to the power of 4
🔧 TOOL CALLED: compute_factorial
   INPUT: n = 7
   OUTPUT: {'status': 'success', 'result': 'The factorial of 7 is 5040'}
🔧 TOOL CALLED: compute_exponential
   INPUT: base = 3.0, exponent = 4.0
   OUTPUT: {'status': 'success', 'result': '3.0 raised to the power of 4.0 is 81.0'}
🤖 Agent Response:
- 7! = 5040
- 3^4 = 81
